In [1]:
### IMPORTING ALL THE REQUIRED LIBRARIES

import numpy as np 
import pandas as pd
import re 
import os
import datetime as dt

In [2]:
## Reading the list of products to be used for search and store operation( This has been made using the file provided by Manish Upreti where the product names were extracted manually )
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']

# Script for Import File

In [76]:


# Taking the file path input from the user 
# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt


file_path = input('Please Enter path for Import file below: ')



if os.path.exists(file_path):
    print('The file does exist')

    #with open(file_path, 'r', encoding="latin-1") as f:
       # lines = f.readlines()
    print(file_path)
    ## Reading the file in excel
    df2 = pd.read_excel(file_path)
    ### Looping through the Product Description column and searching for elements in the Product column and the "2 4 d " AI.
    map_24d = { v: '2 4 d' for v in ['2 4-d', '24 d', '2,4-d', '2 4 d', '2 4- d', '2 4 -d', '2 4d',
       '2, 4 d', '2, 4-d', '2,4 - d', '2,4 d', '2,4- d', '2,4d', '2-4-d',
       '24d', '2-4 d', '24- d', '2-4- d', '2-4d', '24-d', '2, 4- d',
       '2, 4d', '2- 4-d']}


    def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d|_|\.|,|\?|\^|\$|\[|\]|\{|\}'),x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
    df2['Goods Description'] = df2['Goods Description'].map(str)
    
    print("Working on Product Mapping")
    temp = df2['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

    temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
    print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
    df2.loc[np.logical_and(df2['Unit'].gt('GMS'), np.less_equal(df2['Quantity'], 1)), 'Product'] = ''
    df2.loc[df2["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
    temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
    temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
    temp_processed.replace('mancozebe','mancozeb',inplace = True)
    temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
    temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
    temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
    temp_processed.replace('glyphosatee','glyphosate',inplace = True)
    temp_processed.replace('','Others',inplace = True)
    
    df2['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
    df2['Product'] = df2['Product'].fillna('Others')
    
    print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
    # Creating a function for the same
    def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
    df2['suffix'] =  df2['Goods Description'].apply(lambda x: suf(x))
    print("Suffix Creation done")
    #Writing the concerned file
    df2.to_csv('output_file.csv')

    
else:
    
    print('The specified file DOES NOT EXIST!!!')
    

Please Enter path below: C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\October\Import\38_Imp_TY2_Oct-22.xlsx
The file does exist
C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\October\Import\38_Imp_TY2_Oct-22.xlsx
Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [77]:
### Block for altering the entire data into the required format


temp = df2.drop(['Port Code', 'Indian Port', 'Declaration No','Address', 'City', 'CHA No', 'CHA Name','HS Code','Total_Duty_Usd', 'Loading Port','Supplier Address', 'BE Type', 'IEC Phone','IEC Email', 'IEC Contact'],axis = 1)
temp['Product Name'] = temp['Product'] + " " + temp['suffix']
temp = temp.drop('suffix',axis = 1)
temp['Type'] = 'Import'
temp['Year'] = pd.DatetimeIndex(temp['Date']).year
temp['Month'] = pd.DatetimeIndex(temp['Date']).month
temp.rename(columns = {'Importer Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Price Usd':'UNIT PRICE (USD)','Total Value Usd':'Total_VALUE_USD','Inv Currency':'INVOICE CURRENCY','Supplier Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)
temp['ORIGIN/DESTINATION COUNTRY'] = temp['Country']
temp['Company'] = temp['IMPORTER/EXPORTER NAME']


temp = temp.loc[:,["Type","Year","Month","Date","IMPORTER/EXPORTER NAME","ORIGIN/DESTINATION COUNTRY","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Country","Product Name","Company"]]
temp['Product Name'].replace('Others Others','Others',inplace = True)

# Script for Export file

In [74]:

# Taking the file path input from the user 
# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt


file_path = input('Please Enter path for the export file below: ')



if os.path.exists(file_path):
    print('The file does exist')

    #with open(file_path, 'r', encoding="latin-1") as f:
       # lines = f.readlines()
    print(file_path)
    ## Reading the file in excel
    df2 = pd.read_excel(file_path)
    ### Looping through the Product Description column and searching for elements in the Product column and the "2 4 d " AI.
    map_24d = { v: '2 4 d' for v in ['2 4-d', '24 d', '2,4-d', '2 4 d', '2 4- d', '2 4 -d', '2 4d',
       '2, 4 d', '2, 4-d', '2,4 - d', '2,4 d', '2,4- d', '2,4d', '2-4-d',
       '24d', '2-4 d', '24- d', '2-4- d', '2-4d', '24-d', '2, 4- d',
       '2, 4d', '2- 4-d']}


    def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d|_',x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
    df2['Goods Description'] = df2['Goods Description'].map(str)
    
    print("Working on Product Mapping")
    temp = df2['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

    temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
    print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
    df2.loc[np.logical_and(df2['Unit'].gt('GMS'), np.less_equal(df2['Quantity'], 1)), 'Product'] = ''
    df2.loc[df2["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
    temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
    temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
    temp_processed.replace('mancozebe','mancozeb',inplace = True)
    temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
    temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
    temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
    temp_processed.replace('glyphosatee','glyphosate',inplace = True)
    temp_processed.replace('','Others',inplace = True)
    
    df2['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
    df2['Product'] = df2['Product'].fillna('Others')
    
    print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
    # Creating a function for the same
    def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
    df2['suffix'] =  df2['Goods Description'].apply(lambda x: suf(x))
    print("Suffix Creation done")
    
   

    
else:
    
    print('The specified file DOES NOT EXIST!!!')
    
    

Please Enter path below: C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\October\Export\38_Exp_TY2_Oct-22.xlsx
The file does exist
C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\October\Export\38_Exp_TY2_Oct-22.xlsx
Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [75]:
### Block for altering the entire data into the required format

temp2 = df2.drop(['Declaration No', 'HS Four Digit', 'HS Code','Destination Port','Indian Port','Address', 'City','Consinee Address','Port Code', 'IEC Phone', 'IEC Email', 'IEC Contact'],axis = 1)
temp2['Product Name'] = temp2['Product'] + " " + temp2['suffix']
temp2 = temp2.drop('suffix',axis = 1)
temp2['Type'] = 'Export'
temp2['Year'] = pd.DatetimeIndex(temp2['Date']).year
temp2['Month'] = pd.DatetimeIndex(temp2['Date']).month
temp2.rename(columns = {'Exporter Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Value Usd':'UNIT PRICE (USD)','Fob Usd':'Total_VALUE_USD','Currency':'INVOICE CURRENCY','Consinee Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)
temp2['ORIGIN/DESTINATION COUNTRY'] = temp2['Country']
temp2['Company'] = temp2['IMPORTER/EXPORTER NAME']

temp2 = temp2.loc[:,["Type","Year","Month","Date","IMPORTER/EXPORTER NAME","ORIGIN/DESTINATION COUNTRY","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Country","Product Name","Company"]]
temp2['Product Name'].replace('Others Others','Others',inplace = True)


In [178]:
## Concatinating the import and export file to make it as the final file 
final_file = pd.concat([temp,temp2])
final_file['Date'] = pd.to_datetime(final_file['Date'],errors = 'coerce')
final_file['Date'] = final_file['Date'].dt.normalize()

In [161]:

## Reading the file that has been compiled for all but the previous month 
file_previous_month = pd.read_excel(r'IN ExIM Data 2019 Onwards-Key AIs.xlsx',sheet_name = 'Data')

In [162]:
##Making transformations on the previous file to stack up data 

file_previous_month.columns = file_previous_month.iloc[0]
file_previous_month = file_previous_month.iloc[:,0:-2]
file_previous_month = file_previous_month[file_previous_month.isnull().sum(axis = 1) < 4]
file_previous_month = file_previous_month.iloc[1:,:]

file_previous_month['Date'] = pd.to_datetime(file_previous_month['Date'],errors = 'coerce')
file_previous_month['Date'] = file_previous_month['Date'].dt.normalize()

In [180]:
## Concatinating previous month and this months data 
compiled_file = pd.concat([file_previous_month,final_file])
### Changing the DATE column from Object to datetime format 


In [ ]:
pd.read_excel

In [181]:
### Writing the file to the desired location
compiled_file.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\main.xlsx",sheet_name = 'Data',index = False)